In [2]:
from __future__ import division 
import numpy as np
import matplotlib
from math import *
from matplotlib import pyplot as plt
from scipy import special as sp
from scipy import linalg
import scipy.io as sio
from math import *

%matplotlib notebook

In [15]:
#Parameters

num = 50 #How many times to repeat for statistics
t_delays = 30; #Number of time delays
offset = 10; #Offset in pixels x 2
    

#Second pulse with angular misalignment or imcomplete overlap with the first pulse
A = 0.5 #Intensity
FWHM = 10000 #Actually is square root of this number * 2.35
energy = 0.12398 # 10 keV in nm
angle = 30*10**-6 # 60 urad angular mismatch

#Size of detector
xpix = 1024;
ypix = 1024; 

In [16]:
def noisey(image):
    noisy = np.random.poisson(image)
    return noisy


for j in range(0,num):

    t = np.arange(0,t_delays);
    c_t = np.exp(-(t/200)**2);
    phasors= np.zeros([xpix,ypix, t_delays], dtype = complex);
    phasors[:,:,0] = np.random.random([xpix,ypix]);
    mu, sigma = 0, 1 # mean and standard deviation
    G = np.zeros([xpix,ypix]);


    G[:,:] = np.random.normal(mu, sigma, [xpix,ypix]);

    # Method for generating phasors with changing correlation to mimic dynamics
    for i in range(1,phasors.shape[2]):
        phasors[:,:,i] = phasors[:,:,i-1]+G[:,:]*np.sqrt(np.log(c_t[i-1]) - np.log(c_t[i]));
    
    particles = np.zeros([xpix,ypix,t_delays], dtype = complex);
    oldfield = np.zeros([xpix,ypix,t_delays], dtype = complex);
    newfield = np.zeros([xpix,ypix,t_delays], dtype = complex);
    
    oldfield2 = np.zeros([xpix,ypix,t_delays], dtype = complex);
    newfield2 = np.zeros([xpix,ypix,t_delays], dtype = complex);
    
    imageintensity = np.zeros([xpix,ypix,t_delays]);
    imageintensity2 = np.zeros([xpix,ypix,t_delays]);

    #Circular or Gaussian pulse
    x = np.arange(1,xpix+1);
    y = np.arange(1,ypix+1);
    xx, yy = np.meshgrid(x, y, sparse = False);
    xx = xx - np.size(x)/2;
    yy = yy - np.size(y)/2;

    # Linear, undistorted pulse
    illumination = np.zeros([xpix,ypix]);
    illumination[:,:] = A*np.exp(-0.5*((xx-offset)**2+yy**2)/FWHM);
    
    # F = np.exp(3.1*1j*2*pi*(xx+offset)/1000); #offset by 60mrad
    F = np.exp((2*pi*1j/energy)*angle*6.383*(xx+offset))
    illumination2 = np.zeros([xpix,ypix]);
    illumination2[:,:] = A*np.exp(-0.5*((xx+offset)**2+yy**2)/FWHM);
    illumination2 = F*illumination2;

    #Image intensity
    for i in range(0, particles.shape[2]):

        particles[:,:,i] = np.exp(2*pi*1j*phasors[:,:,i]);
        
        oldfield[:,:,i] = illumination[:,:]*np.fft.fft2(particles[:,:,i]);
        newfield[:,:,i] = np.fft.ifft2(oldfield[:,:,i]);
        imageintensity[:,:,i] = (np.abs(newfield[:,:,i]))**2;
        
        oldfield2[:,:,i] = illumination2[:,:]*np.fft.fft2(particles[:,:,i]);
        newfield2[:,:,i] = np.fft.ifft2(oldfield2[:,:,i]);
        imageintensity2[:,:,i] = (np.abs(newfield2[:,:,i]))**2;#Final intensity distribution on image plane 

#Initialize final detector image of sum of two speckle patterns
    off_img = np.zeros([xpix,ypix,t_delays-1])
    norm_img = np.zeros([xpix,ypix,t_delays-1])

    for i in range(0,imageintensity.shape[2]-1):
        off_img[:,:,i] = imageintensity[:,:,0]+imageintensity2[:,:,i];
        off_img[:,:,i] = noisey(off_img[:,:,i])
        
        norm_img[:,:,i] = imageintensity[:,:,0]+imageintensity[:,:,i];
        norm_img[:,:,i] = noisey(norm_img[:,:,i])

    sio.savemat('Simulation/overlap_20pix_offset_30urad/scan' + str(j) +'.mat', {'imagestack':off_img[:,:,:], 'normstack':norm_img[:,:,:]}, do_compression=True)